In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

/home/zhangjianhui/anaconda3/envs/Syft/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhangjianhui/anaconda3/envs/Syft/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhangjianhui/anaconda3/envs/Syft/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhangjianhui/anaconda3/envs/

In [2]:
hook = sy.TorchHook(torch) 

w1 = sy.VirtualWorker(id="worker1", hook=hook)
w2 = sy.VirtualWorker(id="worker2", hook=hook)
c = sy.VirtualWorker(id="c", hook=hook)

In [3]:
w = torch.Tensor([[2., 0.], [0., 2.]]).send(w1)
x = torch.Tensor([1., 1.]).send(w2)

print(w1._objects)
print(w2._objects)

{30229242768: tensor([[2., 0.],
        [0., 2.]])}
{74348889784: tensor([1., 1.])}


In [4]:
w_ptr = w.fix_precision().share(w1, w2, crypto_provider=c)
x_ptr = x.fix_precision().share(w1, w2, crypto_provider=c)

print(w_ptr)
print(x_ptr)

(Wrapper)>[PointerTensor | me:51857177823 -> worker1:50430961851]
(Wrapper)>[PointerTensor | me:18193930528 -> worker2:53728090148]


In [5]:
print(w1._objects)
print(w2._objects)

{57946733457: tensor([[1993861354790943884, 1756147055626835931],
        [2265946151417149017, 3382372701662400204]]), 50430961851: (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:91760375681 -> worker1:57946733457]
	-> [PointerTensor | me:47512523568 -> worker2:22502876463]
	*crypto provider: c*, 54958947164: tensor([2645418134025052650, 2480803711500147970])}
{22502876463: tensor([[2617824663636446020, 2855538962800551973],
        [2345739867010238887, 1229313316764989700]]), 91523947041: tensor([1966267884402336254, 2130882306927240934]), 53728090148: (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:22477973684 -> worker1:54958947164]
	-> [PointerTensor | me:31620392541 -> worker2:91523947041]
	*crypto provider: c*}


In [6]:
w_enc = w_ptr.get()
x_enc = x_ptr.get()

In [7]:
ans = w_enc.mm(x_enc)

print(ans)

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:43907465904 -> worker1:94974280308]
	-> [PointerTensor | me:49726179722 -> worker2:90435654961]
	*crypto provider: c*


In [8]:
print(ans.get().float_precision())

(Wrapper)>FixedPrecisionTensor>tensor([2000, 2000])
